In [100]:
# Mongo 연결
from pymongo import MongoClient as mc
import random
import pprint as pp

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)

keti_pr_db = client.keti_pattern_recognition
household_col = keti_pr_db.household_info

total = household_col.count()
db_datas = list(household_col.\
                find().skip(random.randrange(0,total)).limit(1))[0]
# db_datas = household_col.\
#                 find_one({
#                     "uid": "아파트4-103-606"
#                 })

client.close()

# require module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

sns.set(rc={'figure.figsize': (15.7, 13.27)})
plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

# custom module
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import custom.supports as sp
import custom.mapgenerator as mg
import custom.dateutils as du

SEASONS = {
    "봄" : [3, 4, 5],
    "여름" : [6, 7, 8],
    "가을" : [9, 10, 11],
    "겨울" : [12, 1, 2]
}

SEASONSARR = ['봄','여름','가을','겨울']

'''
 0 : 봄
 1 : 여름
 2 : 가을
 3 : 겨울
'''

DAYARR = ["월", "화", "수", "목", "금", "토", "일"]

uid, timeslot = db_datas['uid'], db_datas['timeslot']

print("{} pattern recognition".format(uid))

datelist = [
    dt.strptime(ts['time'], "%Y-%m-%d T%H:%M %z").date()
    for ts in timeslot
]
datelist = list(set(datelist))
datelist.sort()

ts_datas = {}
start_idx = 0
end_idx = 96
enl = 1000

for date in datelist:
    ts_datas[date] = [ ts['power'] * enl for ts in timeslot[start_idx:end_idx]]
    start_idx = end_idx
    end_idx = end_idx + 96

ts_datas = pd.DataFrame(ts_datas).T
datas = ts_datas.reset_index().copy()

datas.rename(columns={"index": "date"}, inplace=True)
datas['date'] = pd.to_datetime(datas['date'])
datas['month'] = [dt.month for dt in datas['date']]
datas = [
    datas[(datas['month'].isin(SEASONS[season]))].copy()
    for season in SEASONSARR
]

bak_datas = datas.copy()

season_datas = {
    "봄": datas[0].copy(),
    "여름": datas[1].copy(),
    "가을": datas[2].copy(),
    "겨울": datas[3].copy()
}

season_datas["봄"]

<ipython-input-100-d6edea75253d>:12: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  total = household_col.count()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
아파트3-103-1018 pattern recognition


,date,0,1,2,3,4,5,6,7,8,...,87,88,89,90,91,92,93,94,95,month
0,2018-05-01,52.0,52.0,36.0,45.0,60.0,33.0,47.0,70.0,23.0,...,60.0,46.0,38.0,56.0,34.0,41.0,58.0,33.0,43.0,5
1,2018-05-02,44.0,44.0,44.0,76.0,64.0,38.0,47.0,15.0,30.0,...,42.0,34.0,57.0,41.0,35.0,56.0,38.0,38.0,57.0,5
2,2018-05-03,38.0,38.0,55.0,33.0,42.0,56.0,49.0,78.0,49.0,...,61.0,68.0,53.0,62.0,59.0,56.0,44.0,31.0,60.0,5
3,2018-05-04,40.0,40.0,55.0,48.0,30.0,76.0,76.0,15.0,35.0,...,39.0,38.0,56.0,36.0,40.0,57.0,32.0,45.0,55.0,5
4,2018-05-05,40.0,40.0,51.0,31.0,48.0,52.0,64.0,67.0,29.0,...,26.0,50.0,24.0,41.0,32.0,32.0,41.0,33.0,41.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2019-04-26,57.0,68.0,63.0,62.0,61.0,61.0,35.0,51.0,70.0,...,56.0,45.0,69.0,57.0,44.0,69.0,61.0,45.0,62.0,4
361,2019-04-27,55.0,36.0,86.0,73.0,36.0,59.0,74.0,49.0,60.0,...,57.0,51.0,32.0,58.0,46.0,64.0,62.0,49.0,64.0,4
362,2019-04-28,81.0,81.0,39.0,67.0,59.0,58.0,36.0,70.0,69.0,...,44.0,67.0,58.0,43.0,69.0,60.0,44.0,68.0,59.0,4
363,2019-04-29,44.0,66.0,57.0,38.0,71.0,56.0,29.0,43.0,43.0,...,48.0,66.0,47.0,49.0,78.0,49.0,51.0,68.0,49.0,4


In [101]:
# merge timeslot
timeslot = 3
merge_size = timeslot * 4
merge_datas = season_datas["봄"].copy()
merge_datas = merge_datas[
    merge_datas.columns.difference(['month', 'date'])
]
merge_datas.index = season_datas['봄']['date']

tmp = pd.DataFrame()
start_idx = 0
end_idx = 96
for date in merge_datas.index:
    in_arr = []
    og_arr = merge_datas.loc[date].values
    for merging in range(0, int(end_idx / merge_size)):
        in_arr.append(og_arr[
            (merging) * merge_size
            :
            (merging + 1) * merge_size
        ].sum())
#     logging
#     print("date end : {}, size: {}".format(date, len(in_arr)))
    
    tmp[date] = in_arr
merge_datas = tmp.copy()

merge_datas

,2018-05-01,2018-05-02,2018-05-03,2018-05-04,2018-05-05,2018-05-06,2018-05-07,2018-05-08,2018-05-09,2018-05-10,...,2019-04-21,2019-04-22,2019-04-23,2019-04-24,2019-04-25,2019-04-26,2019-04-27,2019-04-28,2019-04-29,2019-04-30
0,540.0,480.0,530.0,492.0,518.0,415.0,472.0,435.0,380.0,741.0,...,688.0,380.0,659.0,472.0,730.0,661.0,744.0,688.0,578.0,561.0
1,369.0,365.0,295.0,300.0,387.0,385.0,299.0,321.0,290.0,298.0,...,375.0,411.0,539.0,333.0,414.0,253.0,266.0,293.0,356.0,419.0
2,302.0,356.0,375.0,288.0,293.0,223.0,289.0,219.0,289.0,285.0,...,323.0,318.0,313.0,232.0,318.0,216.0,220.0,242.0,303.0,320.0
3,307.0,311.0,390.0,342.0,289.0,271.0,546.0,206.0,414.0,284.0,...,315.0,397.0,503.0,225.0,340.0,218.0,428.0,231.0,305.0,311.0
4,430.0,434.0,739.0,579.0,507.0,214.0,625.0,567.0,478.0,419.0,...,796.0,573.0,695.0,760.0,435.0,987.0,608.0,815.0,537.0,342.0
5,733.0,433.0,606.0,380.0,602.0,726.0,533.0,493.0,472.0,579.0,...,783.0,766.0,792.0,780.0,381.0,785.0,375.0,634.0,885.0,513.0
6,419.0,797.0,501.0,658.0,425.0,599.0,439.0,528.0,384.0,421.0,...,682.0,1070.0,641.0,727.0,682.0,780.0,487.0,605.0,799.0,660.0
7,541.0,531.0,623.0,533.0,428.0,617.0,431.0,416.0,330.0,500.0,...,544.0,671.0,561.0,754.0,669.0,682.0,627.0,683.0,675.0,558.0


In [139]:
from sklearn.cluster import KMeans

means_datas = merge_datas.T.copy()
means_points = means_datas.values

kmeans = KMeans(
    n_clusters=14,
    algorithm="full",
    verbose=1)
kmeans.fit_transform(means_points)

cluster_info = pd.DataFrame()
cluster_info['label'] = kmeans.labels_
cluster_info.index = means_datas.index

cluster_info

Initialization complete
Iteration 0, inertia 3928041.0.
Iteration 1, inertia 3015398.500550672.
Iteration 2, inertia 2851858.1441369047.
Iteration 3, inertia 2774648.1032467536.
Converged at iteration 3: strict convergence.
Initialization complete
Iteration 0, inertia 4353245.0.
Iteration 1, inertia 3114018.4479708704.
Iteration 2, inertia 2929393.0490319203.
Iteration 3, inertia 2911592.9358974355.
Converged at iteration 3: strict convergence.
Initialization complete
Iteration 0, inertia 4003287.0.
Iteration 1, inertia 3094936.1990138064.
Iteration 2, inertia 2981293.179113462.
Iteration 3, inertia 2952882.616724941.
Iteration 4, inertia 2927654.947589138.
Iteration 5, inertia 2915437.416724941.
Converged at iteration 5: strict convergence.
Initialization complete
Iteration 0, inertia 4819248.0.
Iteration 1, inertia 3196935.57556458.
Iteration 2, inertia 2951391.9930952387.
Iteration 3, inertia 2872370.0420634924.
Converged at iteration 3: strict convergence.
Initialization complete
I

,label
2018-05-01,7
2018-05-02,4
2018-05-03,0
2018-05-04,2
2018-05-05,7
...,...
2019-04-26,5
2019-04-27,2
2019-04-28,5
2019-04-29,3


In [140]:
map_datas = pd.DataFrame()

for date in merge_datas:
    tmp = pd.DataFrame()
    tmp['timeslot'] = range(0, 8)
    tmp['data'] = means_datas.loc[date].values
    tmp['date'] = date
    tmp['label'] = cluster_info.loc[date]['label']
    
    map_datas = pd.concat([tmp, map_datas])

_map = sns.FacetGrid(map_datas, col="label", col_wrap=3, height=30)    
_map.map(sns.lineplot, "timeslot", "data", "date", lw=0.7)

In [141]:
cluster_pattern = {}

for num in map_datas['label'].unique():
    date_list = map_datas[
        map_datas['label'] == num
    ]['date'].unique()
    cluster_pattern[num] = merge_datas.T.loc[date_list].mean().values
        
cluster_pattern

{11: array([555.42857143, 383.14285714, 337.14285714, 340.14285714,
        373.14285714, 625.28571429, 694.        , 578.        ]),
 3: array([635.16666667, 467.16666667, 335.66666667, 398.16666667,
        598.        , 727.16666667, 684.83333333, 633.5       ]),
 5: array([627.25, 313.5 , 253.25, 247.25, 839.5 , 745.5 , 698.5 , 665.75]),
 2: array([518.54545455, 291.36363636, 310.54545455, 392.        ,
        588.72727273, 452.09090909, 513.81818182, 545.18181818]),
 9: array([595.45454545, 372.36363636, 377.72727273, 389.72727273,
        387.36363636, 386.63636364, 613.18181818, 639.18181818]),
 10: array([ 380.,  411.,  318.,  397.,  573.,  766., 1070.,  671.]),
 12: array([564.33333333, 527.66666667, 392.66666667, 293.66666667,
        311.66666667, 928.66666667, 667.33333333, 607.        ]),
 8: array([543.77777778, 343.33333333, 352.33333333, 398.66666667,
        555.77777778, 544.        , 712.55555556, 606.66666667]),
 0: array([514.        , 332.88888889, 324.88888889, 

In [144]:
from scipy.spatial import distance

# Mean_Pattern
mean_pattern = merge_datas.T.copy().mean().values

# TSS
TSS = 0
for date in merge_datas:
    TSS += distance.euclidean(mean_pattern,
                       merge_datas[date].values)

TSS

WSS = 0
for date in merge_datas:
    WSS += distance.euclidean(
        map_datas[
            map_datas['date'] == date
        ]['data'].values        
        ,
        kmeans.cluster_centers_[
            cluster_info.loc[date]['label']
        ]
    )

ECV = (TSS - WSS) / TSS * 100
print("TSS: {}, WSS: {}, ECV: {}".format(TSS,WSS,ECV))

TSS: 33562.20842129181, WSS: 15067.032096410965, ECV: 55.10714936490157


In [198]:
import random

def means_run(K=3):
    cluster_dict = {}
    cluster_index = []
    
    row_datas = merge_datas.T.copy()
    init_k = random.randint(0,len(row_datas.index))
    init_cluster_index = row_datas.index[init_k]
    init_cluster = row_datas.iloc[init_k]
    cluster_dict[0] = init_cluster
    cluster_index.append(init_cluster_index)
#     print(init_cluster_index,init_cluster)
    
    for seq in range(1, K):
        dis_arr = ['dis_{}'.format(idx) for idx in range(0,seq)]
        dis_check = pd.DataFrame(columns=dis_arr)
        for date in row_datas.index:
            if date not in cluster_index:
                dis_check.loc[date] = distance.euclidean(
                    init_cluster.values,
                    row_datas.loc[date].values
                )
        cluster_index.append(dis_check.sort_values(by=dis_arr).index[0])

        print(dis_check.sort_values(by=dis_arr, ascending=False).index[0],
             dis_check.sort_values(by=dis_arr, ascending=False).iloc[0].values
             )
        cluster_dict[seq]\
            = row_datas.loc[dis_check.sort_values(by=dis_arr, ascending=False).index[0]].values
    return cluster_dict

dis_check = means_run()
dis_check
# index=dis_check.sort_values(by=['dis']).index[0]
# index

2019-03-26 00:00:00 [846.40002363]
2019-03-26 00:00:00 [846.40002363 846.40002363]


{0: 0    472.0
 1    299.0
 2    289.0
 3    546.0
 4    625.0
 5    533.0
 6    439.0
 7    431.0
 Name: 2018-05-07 00:00:00, dtype: float64,
 1: array([167., 169., 282., 171., 166., 183., 181., 174.]),
 2: array([167., 169., 282., 171., 166., 183., 181., 174.])}